<a href="https://colab.research.google.com/github/arndmghsh/TopicModelling-GaussianLDA-GibbsSampler/blob/master/glda_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
from scipy.stats import dirichlet, multivariate_normal, multinomial
from utils import *
import heapq

In [0]:
class GaussianLDA(object):
    """Object that encapsulates Gaussian LDA algorithm"""
    def __init__(self, n_topics):
        #Set up variables used throughout process.
        self.n_topics = n_topics
        self.embeddings, self.word2index = load_embeddings()
        self.index2word = {self.word2index[w]:w for w in self.word2index}
        self.corpus = load_corpus()
        print(self.corpus.shape)

        self.D = self.corpus.shape[0]
        self.V = self.corpus.shape[1]
        self.M = 50
        #Randomly assign words to topics
        self.z_di = {}
        #Count document / topic coocurrences 
        self.Nk = np.zeros(n_topics)
        self.vk_bar = np.zeros((n_topics, self.M))
        self.n_kd = np.zeros((n_topics, self.D))
        for d in range(self.D):
            for i in range(self.V):
                self.z_di['d'+str(d)+'i'+str(i)] = []
                for w in range(int(self.corpus[d][i])):
                    k = np.random.randint(n_topics)
                    self.z_di['d'+str(d)+'i'+str(i)].append(k)
                    self.Nk[k] += 1
                    self.vk_bar[k] += self.embeddings[i]
                    self.n_kd[k][d] += 1

        #Set up prior parameters
        self.mu = np.zeros(self.M)
        # self.mu = np.sum(self.embeddings, axis=0)/self.V
        
        self.kappa = 0.01
        self.Sigma = np.identity(self.M)
        self.nu = self.M
        # for each topic
        self.mu_k = np.zeros((n_topics, self.M))
        self.nu_k = np.zeros(n_topics)
        self.kappa_k = np.zeros(n_topics)
        
        #Set up topic parameters      
        self.alpha = 10*np.ones(n_topics)
        # Update topic parameters
        for k in range(n_topics):
            self.update_topic_parameters(k)

    def log_likelihood(self):
        #Return the log-likelihood of the joint assignment of topics and words under G-LDA
        logll = 0
        for d in range(self.D):
            # calculate p_theta_d for each document only once
            theta_d = self.n_kd[:,d]/(np.sum(self.n_kd[:,d]))
            log_p_theta_d = dirichlet.logpdf(theta_d, self.alpha)
            logll += log_p_theta_d
            for i in range(self.V):
                for w in range(int(self.corpus[d][i])):
                    # Topic assigned
                    k = self.z_di['d'+str(d)+'i'+str(i)][w]
                    # # calculate p_theta_d
                    # theta_d = self.n_kd[:,d]/(np.sum(self.n_kd[:,d]))
                    # log_p_theta_d = dirichlet.logpdf(theta_d, self.alpha)

                    # # calculate p_mu_k
                    # log_p_mu_k = multivariate_normal.logpdf(self.mu_k[k], self.mu, \
                    #                 (1/self.kappa)*np.identity(self.M))
                    
                    # calculate p_zk
                    log_p_zk = np.log(theta_d[k])
                    # calculate p_v_di
                    log_p_v_di = multivariate_normal.logpdf(self.embeddings[i], self.mu_k[k], \
                                                                        np.identity(self.M))
                    # logll += log_p_theta_d + log_p_mu_k + log_p_zk + log_p_v_di
                    logll +=  log_p_zk + log_p_v_di
        
        # calculate p_mu_k for each topic only once
        for k in range(self.n_topics):
            log_p_mu_k = multivariate_normal.logpdf(self.mu_k[k], self.mu, \
                                    (1/self.kappa)*np.identity(self.M))
            logll += log_p_mu_k
        
        return logll

    def update_topic_parameters(self, k):
        #Update topic parameters for topic k
        self.kappa_k[k] = self.kappa + self.Nk[k]
        self.nu_k[k] = self.nu + self.Nk[k]
        self.mu_k[k] = (self.kappa*self.mu + self.vk_bar[k])/(self.kappa_k[k])
        
    def print_top(self, n=10):

        log_probs = np.zeros((self.V, self.n_topics))
        for i in range(self.V):
            for k in range(self.n_topics):
                degs_freedom = self.nu_k[k] - self.M + 1
                log_probs[i,k] = multivariate_t_distribution(self.embeddings[i], self.mu_k[k,:], degs_freedom, self.M) 

        log_probs -= np.max(log_probs, axis=1).reshape(-1,1)
        log_probs = np.exp(log_probs)
        log_probs /= np.sum(log_probs, axis=1).reshape(-1,1)
    
        top_words_order = np.argsort(-1*log_probs, axis=0)
        top_words = []
        for k in range(self.n_topics):
            top_words_k = [self.index2word[indx] for indx in top_words_order[0:n,k]]
            top_words.append(top_words_k)
            print(top_words_k)
        
        return None

    def sample(self):
        #Do calculation of parameters and sample from posterior
        for d in range(self.D):
            for i in range(self.V):
                for w in range(int(self.corpus[d][i])):
                    # current topic
                    k = self.z_di['d'+str(d)+'i'+str(i)][w]
                    # Adjust statistics
                    self.Nk[k]-=1
                    self.vk_bar[k] -= self.embeddings[i]
                    self.n_kd[k][d] -= 1
                    # Update topic parameters
                    self.update_topic_parameters(k)
                    # Calculate full conditional for each topic
                    maxlogp = float('-inf')
                    ptilde_k = np.zeros(self.n_topics)
                    for k in range(self.n_topics):
                        ptilde_k[k] = ((self.n_kd[k][d]+ self.alpha[k])*
                                            multivariate_t_distribution(self.embeddings[i,:], 
                                                self.mu_k[k,:], self.nu_k[k]-self.M+1, self.M))
                        maxlogp = max(maxlogp, ptilde_k[k])
                    # Normalize to get the Prob dist over topics
                    for k in range(self.n_topics):
                        ptilde_k[k]-=maxlogp
                    # Convert Log to Linear scale
                    ptilde_k = np.exp(ptilde_k)
                    ptilde_k+= 1e-7
                    ptilde_k /= np.sum(ptilde_k)
                    # Assign the topic sampled from this multinomial over k
                    new_k = np.argmax(multinomial.rvs(1, ptilde_k))
                    self.z_di['d'+str(d)+'i'+str(i)][w] = new_k
                    # Adjust statistics
                    self.Nk[new_k] += 1
                    self.vk_bar[new_k] += self.embeddings[i]
                    self.n_kd[new_k][d] += 1

        return None

In [18]:
glda = GaussianLDA(5)
n_itr = 50
for itr in range(n_itr):
    glda.sample()
    print(itr)
    if itr%10==0:
        logll = glda.log_likelihood()
        print(itr, ':', logll)
logll = glda.log_likelihood()
print(itr, ':', logll)

glda.print_top()

Loading embeddings...
(3603, 50)
Loading corpus...
(596, 3603)
0
0 : -2840452.0224716566
1
2
3
4
5
6
7
8
9
10
10 : -2791240.463436879
11
12
13
14
15
16
17
18
19
20
20 : -2788309.011919851
21
22
23
24
25
26
27
28
29
30
30 : -2787717.6172379516
31
32
33
34
35
36
37
38
39
40
40 : -2787429.79639926
41
42
43
44
45
46
47
48
49
49 : -2787282.5795358713
['research', 'organization', 'organizations', 'institute', 'education', 'sciences', 'agency', 'council', 'cooperation', 'department']
['informatik', 'obo', 'wisc', 'exe', 'acsu', 'centris', 'nntp', 'prb', 'txt', 'cco']
['vehicles', 'cars', 'vehicle', 'truck', 'tanks', 'planes', 'engine', 'passenger', 'engines', 'metal']
['absolutely', 'honest', 'sense', 'feel', 'understand', 'reasonable', 'things', 'circumstances', 'deserve', 'terribly']
['mike', 'scored', 'coach', 'chris', 'steve', 'james', 'brian', 'kevin', 'john', 'joe']
